## Animacy hierarchy classification inter-annotator agreement

2023

This notebook contains the code for "Experiment 2: introducing an animacy hierarchy" in the article "Identifying visual depictions of animate entities in narrative comics: an annotation study 

##### Imports and libraries

In [1]:

import json # json library
import os # for list directories 

import numpy as np 
import math
import matplotlib.pyplot as plt

import krippendorff


##### Read-in all annotations, per story, in JSON format

In [11]:
# Story 1

f1_1 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story1/Processed/participant1.json")
f2_1 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story1/Processed/participant2.json")
f3_1 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story1/Processed/participant3.json")
f4_1 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story1/Processed/participant4.json")
f5_1 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story1/Processed/participant5.json")

participant1_story1 = json.load(f1_1)
participant2_story1 = json.load(f2_1)
participant3_story1 = json.load(f3_1)
participant4_story1 = json.load(f4_1)
participant5_story1 = json.load(f5_1)


In [12]:
story1 = {"participant1" : participant1_story1,
          "participant2" : participant2_story1,
          "participant3" : participant3_story1,
          "participant4" : participant4_story1,
          "participant5" : participant5_story1}



In [13]:
# Story 2

f1_2 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story2/Processed/participant1.json")
f2_2 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story2/Processed/participant2.json")
f3_2 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story2/Processed/participant3.json")
f4_2 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story2/Processed/participant4.json")
f5_2 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story2/Processed/participant5.json")

participant1_story2 = json.load(f1_2)
participant2_story2 = json.load(f2_2)
participant3_story2 = json.load(f3_2)
participant4_story2 = json.load(f4_2)
participant5_story2 = json.load(f5_2)


In [14]:
story2 = {"participant1" : participant1_story2,
          "participant2" : participant2_story2,
          "participant3" : participant3_story2,
          "participant4" : participant4_story2,
          "participant5" : participant5_story2}


In [15]:
# Story 3

f1_3 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story3/Processed/participant1.json")
f2_3 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story3/Processed/participant2.json")
f3_3 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story3/Processed/participant3.json")
f4_3 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story3/Processed/participant4.json")
f5_3 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story3/Processed/participant5.json")

participant1_story3 = json.load(f1_3)
participant2_story3 = json.load(f2_3)
participant3_story3 = json.load(f3_3)
participant4_story3 = json.load(f4_3)
participant5_story3 = json.load(f5_3)




In [16]:
story3 = {"participant1" : participant1_story3,
          "participant2" : participant2_story3,
          "participant3" : participant3_story3,
          "participant4" : participant4_story3,
          "participant5" : participant5_story3}


In [17]:
# Story 4

f1_4 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story4/Processed/participant1.json")
f2_4 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story4/Processed/participant2.json")
f3_4 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story4/Processed/participant3.json")
f4_4 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story4/Processed/participant4.json")
f5_4 = open("Annotations_Prolific/Animacy_Type_and_Reference_Story4/Processed/participant5.json")

participant1_story4 = json.load(f1_4)
participant2_story4 = json.load(f2_4)
participant3_story4 = json.load(f3_4)
participant4_story4 = json.load(f4_4)
participant5_story4 = json.load(f5_4)



In [18]:
story4 = {"participant1" : participant1_story4,
          "participant2" : participant2_story4,
          "participant3" : participant3_story4,
          "participant4" : participant4_story4,
          "participant5" : participant5_story4}


##### Krippendorff's Alpha Function

In [19]:
# Krippendorff's Alpha - Inter-rater agreement measure
# Code from https://github.com/grrrr/krippendorff-alpha/blob/master/krippendorff_alpha.py

def nominal_metric(a, b):
    return a != b

def interval_metric(a, b):
    return (a-b)**2

def ratio_metric(a, b):
    return ((a-b)/(a+b))**2

def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Python implementation of Krippendorff's alpha -- inter-rater reliability
    (c)2011-17 Thomas Grill (http://grrrr.org)
    Python version >= 2.4 required

    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to 
    coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.


##### Function to organize annotation data

In [25]:

def get_info_per_participant(participant):
    
    references_list = [] # Only references given in the main ref label box
    references_list_with_sameLabels = [] # Reference in the main box plus sameAsOtherLabel box 
    animacyType_list = []
    animacyType_list_converted_num = []
    same_as_other_label_values = []
    for page, page_info in participant.items():
        for panel, panel_info in page_info.items():
            agents = panel_info["Agents"]
            for agent, agent_info in agents.items():
                ref_label = agent_info["ref_label"]
                references_list.append(ref_label)
                if (ref_label != "T"):
                    isSameAsOtherLabelValue = agent_info["isSameAsOtherLabel"]
                    if (isSameAsOtherLabelValue == True):
                        #print("isSameAsOtherLabelValue: True")
                        same_as_other_label_values.append(1)
                        sameAsOtherLabelRef = agent_info["isSameAsOtherLabelRef"]
                        #print((agent_info["ref_label"], sameAsOtherLabelRef))
                        references_list_with_sameLabels.append(agent_info["ref_label"]+ "/"+ sameAsOtherLabelRef)
                    else:
                        same_as_other_label_values.append(0)
                else:
                    same_as_other_label_values.append(0)
                animacyType_list.append(agent_info["animacyType"])
                if (agent_info["animacyType"] == "humanAnimate"):
                    animacyType_list_converted_num.append(1.0)
                if (agent_info["animacyType"] == "humanLikeAnimate"):
                    animacyType_list_converted_num.append(2.0)
                if (agent_info["animacyType"] == "nonHumanAnimate"):
                    animacyType_list_converted_num.append(3.0)
                if (agent_info["animacyType"] == "ambiguousAnimate"):
                    animacyType_list_converted_num.append(4.0)
                if (agent_info["animacyType"] == "inanimate"):   
                    animacyType_list_converted_num.append(5.0)
                    
    #print(animacyType_list_converted_num)           
    #print(references_list)
    #print(animacyType_list)
    #print(references_list_with_sameLabels)
    
    return references_list, animacyType_list, animacyType_list_converted_num, references_list_with_sameLabels, same_as_other_label_values  
    
    

##### Calculate All-against-all Krippendorff's alpha scores per story

Story 1

In [26]:
p1_refList1, p1_animacyTypeList1, p1_animacyTypeListNums1, p1_refListSameLabels1, p1_sameLabelsValues1 =  get_info_per_participant(story1["participant1"])
p2_refList1, p2_animacyTypeList1, p2_animacyTypeListNums1, p2_refListSameLabels1, p2_sameLabelsValues1 =  get_info_per_participant(story1["participant2"]) 
p3_refList1, p3_animacyTypeList1, p3_animacyTypeListNums1, p3_refListSameLabels1, p3_sameLabelsValues1 =  get_info_per_participant(story1["participant3"])
p4_refList1, p4_animacyTypeList1, p4_animacyTypeListNums1, p4_refListSameLabels1, p4_sameLabelsValues1 =  get_info_per_participant(story1["participant4"])
p5_refList1, p5_animacyTypeList1, p5_animacyTypeListNums1, p5_refListSameLabels1, p5_sameLabelsValues1 =  get_info_per_participant(story1["participant5"])



In [28]:
kalpha = krippendorff_alpha([p1_animacyTypeListNums1, p2_animacyTypeListNums1, p3_animacyTypeListNums1, p4_animacyTypeListNums1, p5_animacyTypeListNums1], nominal_metric) # get the k alpha for each ind agent
print("Krippendorff's alpha scores: ")
print("nominal: ", kalpha)
print("ordinal: ", krippendorff.alpha([p1_animacyTypeListNums1, p2_animacyTypeListNums1, p3_animacyTypeListNums1, p4_animacyTypeListNums1, p5_animacyTypeListNums1], level_of_measurement="interval"))




Krippendorff's alpha scores: 
nominal:  0.4412709512466947
ordinal:  0.5411357946190836


In [30]:
print("Show the chosen animacy category annotations per participant")
print("\n")

print("Participant 1 *****************")
print(p1_animacyTypeList1)
print("\n")
print("Participant 2 *****************")
print(p2_animacyTypeList1)
print("\n")
print("Participant 3 *****************")
print(p3_animacyTypeList1)
print("\n")
print("Participant 4 *****************")
print(p4_animacyTypeList1)
print("\n")
print("Participant 5 *****************")
print(p5_animacyTypeList1)



Show the chosen animacy category annotations per participant


Participant 1 *****************
['humanAnimate', 'humanAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'ambiguousAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanAnimate', 'humanAnimate', 'humanLikeAnimate', 'ambiguousAnimate', 'ambiguousAnimate', 

Story 2

In [32]:

p1_refList, p1_animacyTypeList, p1_animacyTypeListNums, p1_refListSameLabels, p1_sameLabelsValues =  get_info_per_participant(story2["participant1"])
p2_refList, p2_animacyTypeList, p2_animacyTypeListNums, p2_refListSameLabels, p2_sameLabelsValues =  get_info_per_participant(story2["participant2"]) 
p3_refList, p3_animacyTypeList, p3_animacyTypeListNums, p3_refListSameLabels, p3_sameLabelsValues =  get_info_per_participant(story2["participant3"])
p4_refList, p4_animacyTypeList, p4_animacyTypeListNums, p4_refListSameLabels, p4_sameLabelsValues =  get_info_per_participant(story2["participant4"])
p5_refList, p5_animacyTypeList, p5_animacyTypeListNums, p5_refListSameLabels, p5_sameLabelsValues =  get_info_per_participant(story2["participant5"])


In [34]:
kalpha = krippendorff_alpha([p1_animacyTypeListNums, p2_animacyTypeListNums, p3_animacyTypeListNums, p4_animacyTypeListNums, p5_animacyTypeListNums], nominal_metric) # get the k alpha for each ind agent
print("Krippendorff's alpha scores: ")
print("nominal: ", kalpha)
print("ordinal: ", krippendorff.alpha([p1_animacyTypeListNums, p2_animacyTypeListNums, p3_animacyTypeListNums, p4_animacyTypeListNums, p5_animacyTypeListNums], level_of_measurement="interval"))



Krippendorff's alpha scores: 
nominal:  0.5509392452191573
ordinal:  0.7512710797885729


In [35]:
print("Show the chosen animacy category annotations per participant")
print("\n")

print("Participant 1 *****************")
print(p1_animacyTypeList)
print("\n")
print("Participant 2 *****************")
print(p2_animacyTypeList)
print("\n")
print("Participant 3 *****************")
print(p3_animacyTypeList)
print("\n")
print("Participant 4 *****************")
print(p4_animacyTypeList)
print("\n")
print("Participant 5 *****************")
print(p5_animacyTypeList)



Show the chosen animacy category annotations per participant


Participant 1 *****************
['humanLikeAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanLikeAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate']


Participant 2 *****************
['humanAnimate', 'humanAnimate', 'humanAni

Story 3

In [36]:
p1_refList3, p1_animacyTypeList3, p1_animacyTypeListNums3, p1_refListSameLabels3, p1_sameLabelsValues3 =  get_info_per_participant(story3["participant1"])
p2_refList3, p2_animacyTypeList3, p2_animacyTypeListNums3, p2_refListSameLabels3, p2_sameLabelsValues3 =  get_info_per_participant(story3["participant2"]) 
p3_refList3, p3_animacyTypeList3, p3_animacyTypeListNums3, p3_refListSameLabels3, p3_sameLabelsValues3 =  get_info_per_participant(story3["participant3"])
p4_refList3, p4_animacyTypeList3, p4_animacyTypeListNums3, p4_refListSameLabels3, p4_sameLabelsValues3 =  get_info_per_participant(story3["participant4"])
p5_refList3, p5_animacyTypeList3, p5_animacyTypeListNums3, p5_refListSameLabels3, p5_sameLabelsValues3 =  get_info_per_participant(story3["participant5"])


kalpha = krippendorff_alpha([p1_animacyTypeListNums3, p2_animacyTypeListNums3, p3_animacyTypeListNums3, p4_animacyTypeListNums3, p5_animacyTypeListNums3], nominal_metric) # get the k alpha for each ind agent
print("Krippendorff's alpha scores: ")
print("nominal: ", kalpha)
print("ordinal: ", krippendorff.alpha([p1_animacyTypeListNums3, p2_animacyTypeListNums3, p3_animacyTypeListNums3, p4_animacyTypeListNums3, p5_animacyTypeListNums3], level_of_measurement="interval"))


Krippendorff's alpha scores: 
nominal:  0.3877373111197211
ordinal:  0.55760078387458


In [37]:
print("Show the chosen animacy category annotations per participant")
print("\n")

print("Participant 1 *****************")
print(p1_animacyTypeList3)
print("\n")
print("Participant 2 *****************")
print(p2_animacyTypeList3)
print("\n")
print("Participant 3 *****************")
print(p3_animacyTypeList3)
print("\n")
print("Participant 4 *****************")
print(p4_animacyTypeList3)
print("\n")
print("Participant 5 *****************")
print(p5_animacyTypeList3)


Show the chosen animacy category annotations per participant


Participant 1 *****************
['humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'nonHumanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'hum

Story 4

In [38]:
p1_refList4, p1_animacyTypeList4, p1_animacyTypeListNums4, p1_refListSameLabels4, p1_sameLabelsValues4 =  get_info_per_participant(story4["participant1"])
p2_refList4, p2_animacyTypeList4, p2_animacyTypeListNums4, p2_refListSameLabels4, p2_sameLabelsValues4 =  get_info_per_participant(story4["participant2"])
p3_refList4, p3_animacyTypeList4, p3_animacyTypeListNums4, p3_refListSameLabels4, p3_sameLabelsValues4 =  get_info_per_participant(story4["participant3"])
p4_refList4, p4_animacyTypeList4, p4_animacyTypeListNums4, p4_refListSameLabels4, p4_sameLabelsValues4 =  get_info_per_participant(story4["participant4"])
p5_refList4, p5_animacyTypeList4, p5_animacyTypeListNums4, p5_refListSameLabels4, p5_sameLabelsValues4 =  get_info_per_participant(story4["participant5"])


In [39]:
kalpha = krippendorff_alpha([p1_animacyTypeListNums4,p2_animacyTypeListNums4,p3_animacyTypeListNums4,p4_animacyTypeListNums4,p5_animacyTypeListNums4], nominal_metric) # get the k alpha for each ind agent
print("Krippendorff's alpha scores: ")
print("nominal: ", kalpha)
print("ordinal: ", krippendorff.alpha([p1_animacyTypeListNums4,p2_animacyTypeListNums4,p3_animacyTypeListNums4,p4_animacyTypeListNums4,p5_animacyTypeListNums4], level_of_measurement="interval"))



Krippendorff's alpha scores: 
nominal:  0.7197802197802198
ordinal:  0.7870493107104984


In [41]:
print("Show the chosen animacy category annotations per participant")
print("\n")

print("Participant 1 *****************")
print(p1_animacyTypeList4)
print("\n")
print("Participant 2 *****************")
print(p2_animacyTypeList4)
print("\n")
print("Participant 3 *****************")
print(p3_animacyTypeList4)
print("\n")
print("Participant 4 *****************")
print(p4_animacyTypeList4)
print("\n")
print("Participant 5 *****************")
print(p5_animacyTypeList4)


Show the chosen animacy category annotations per participant


Participant 1 *****************
['humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanAnimate', 'nonHumanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate']


Participant 2 *****************
['humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'ambiguousAnimate', 'humanAnimate', 'humanAnimate', 'nonHumanAnimate', 'humanAnimate', 'nonHumanA